In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import libraries :

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


Load Dataset from kaggle Data set :

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("architsharma01/loan-approval-prediction-dataset")

print("Path to dataset files:", path)

Now check what files are in that path:

In [ ]:
import os

# List the files inside the dataset directory
print("Files:", os.listdir(path))


Let's print the files to verify

In [ ]:

files = os.listdir(path)
print(files)  # Copy the exact file name from here

# Use the correct file name
df = pd.read_csv(os.path.join(path, files[0]))  # or use files[0] if there's only one CSV


In [ ]:
df.isnull().sum()


In [ ]:
print(df.columns.tolist())


In [ ]:
# Normalize column names properly
df.columns = df.columns.str.strip().str.replace(" ", "_").str.lower()

# View cleaned column names
print("Cleaned column names:\n", df.columns.tolist())


In [ ]:
# Step 1: Print actual column names before anything else
print("Original columns:\n", df.columns.tolist())

# Step 2: Clean column names (force standard format)
df.columns = df.columns.str.strip().str.replace(" ", "_").str.lower()
print("\nCleaned columns:\n", df.columns.tolist())

# Step 3: Handle missing values (only for columns that exist)
expected_cols = ['gender', 'married', 'dependents', 'education', 'self_employed',
                 'loanamount', 'loan_amount_term', 'credit_history', 'loan_status']

for col in expected_cols:
    if col in df.columns:
        if df[col].dtype == 'object':
            df[col].fillna(df[col].mode()[0], inplace=True)
        else:
            df[col].fillna(df[col].median(), inplace=True)
    else:
        print(f"⚠️ Column not found: {col}")

# Step 4: Encode categorical columns safely
from sklearn.preprocessing import LabelEncoder

cat_cols = ['gender', 'married', 'education', 'self_employed', 'property_area', 'loan_status']
le = LabelEncoder()
for col in cat_cols:
    if col in df.columns:
        df[col] = le.fit_transform(df[col])
    else:
        print(f"⚠️ Skipping encoding for missing column: {col}")

# Step 5: Final check
print("\n✅ Final check — Missing values:\n", df.isnull().sum())


Feature Selection and Train-Test Split:

In [ ]:


# Drop loan_id (not useful for prediction)
df.drop('loan_id', axis=1, inplace=True)

# Define X (features) and y (target)
X = df.drop('loan_status', axis=1)
y = df['loan_status']

# Train/Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("✅ Training and testing sets created.")
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")


Model Training & Evaluation:

In [ ]:


from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Initialize model
rf_model = RandomForestClassifier(random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on test set
y_pred = rf_model.predict(X_test)

# Evaluate
acc = accuracy_score(y_test, y_pred)
print(f"✅ Accuracy: {acc:.4f}\n")
print("Classification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Rejected', 'Approved'], yticklabels=['Rejected', 'Approved'])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


Feature Importance:

In [ ]:


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Get feature importances from trained model
importances = rf_model.feature_importances_
feature_names = X.columns

# Create DataFrame and sort
feat_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feat_df = feat_df.sort_values(by='Importance', ascending=False)

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feat_df, palette='viridis')
plt.title("Feature Importance (Random Forest)")
plt.xlabel("Importance Score")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()


Hyperparameter Tuning:

In [ ]:


from sklearn.model_selection import GridSearchCV

# Define grid of parameters to try
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# GridSearchCV for tuning
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best parameters and model
print("✅ Best Parameters Found:\n", grid_search.best_params_)

# Evaluate tuned model
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
print(f"\n🔍 Accuracy after tuning: {accuracy_score(y_test, y_pred_best):.4f}")


Save the model to a file:

In [ ]:

import joblib

joblib.dump(best_model, 'loan_model.pkl')
print("✅ Model saved as 'loan_model.pkl'")
